In [212]:
import pandas as pd
import requests

url = 'https://www.slickcharts.com/berkshire-hathaway'

# Adding user-agent headers to imitate a real browser
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36'
}

# Send the request with headers
response = requests.get(url, headers=headers)

# Check if the request was successful
if response.status_code == 200:
    tables = pd.read_html(response.text)
    df = tables[0]
    df = df[['Company', 'Symbol', 'Portfolio%']]
    df.to_csv('BRK-B_company_weightings.csv', index=False)
    print("BRK-B company weightings have been saved to 'BRK-B_company_weightings.csv'.")
else:
    print(f"Failed to retrieve data: {response.status_code}")


BRK-B company weightings have been saved to 'BRK-B_company_weightings.csv'.


/var/folders/wj/3c4wz2j94ldgr3jbcf57vpz00000gn/T/ipykernel_89792/1706769366.py:16: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  tables = pd.read_html(response.text)


In [75]:
import pandas as pd
import yfinance as yf
import time

# Function to get the industry based on the symbol
def get_industry(symbol):
    try:
        company = yf.Ticker(symbol)
        industry = company.info.get('industry', 'Industry not available')
        return industry
    except Exception as e:
        print(f"Error for {symbol}: {e}")
        return 'Industry not available'

# Read the CSV file containing the company symbols
df = pd.read_csv('sp500_company_weightings.csv')

# List to store industries
industry_list = []

# Loop through every 300 rows and process
batch_size = 300
for i in range(0, len(df), batch_size):
    # Process the batch of 300 rows
    for symbol in df['Symbol'][i:i+batch_size]:
        industry = get_industry(symbol)
        industry_list.append(industry)
    
    # After processing 300 rows, sleep for 30 seconds
    if (i + batch_size) < len(df):  # Ensure we don't sleep after the last batch
        print(f"Sleeping for 30 seconds after processing rows {i+1} to {min(i+batch_size, len(df))}")
        time.sleep(30)

# Add the "Industry" column to the DataFrame
df['Industry'] = industry_list

# Save the updated DataFrame with the new "Industry" column to a new CSV file
df.to_csv('sp500_with_industries.csv', index=False)

print("CSV file with industries saved successfully.")


Sleeping for 30 seconds after processing rows 1 to 300
Error for NCLH: Too Many Requests. Rate limited. Try after a while.
CSV file with industries saved successfully.


In [9]:
import yfinance as yf

# Fetch data for BND (Vanguard Total Bond Market ETF)
bnd = yf.Ticker("BND")

# Get the ETF's holdings
holdings = bnd.history(period="1d", actions=False)

# Print some basic information
print(holdings)


                                Open       High        Low  Close   Volume
Date                                                                      
2025-03-24 00:00:00-04:00  73.190002  73.220001  72.980003   73.0  5637000


In [171]:
import yfinance as yf
import pandas as pd
import numpy as np

# Portfolio parameters
tickers = ["GBTC", "SPY", "SOXL", "VUG", "VOO"]
weights = [0.2, 0.2, 0.2, 0.2, 0.2]
start_date = "2019-01-01"
end_date = "2023-12-31"

# Download data
data = yf.download(tickers, start=start_date, end=end_date)

if data is not None and len(data) > 0:
    if "Close" in data:
        try:
            # Extract closing prices
            data = data["Close"]
            
            # Calculate returns
            returns = data.pct_change().dropna()
            
            # Calculate weighted portfolio returns
            weighted_returns = (returns * weights).sum(axis=1)
            
            # Cumulative returns calculation
            cumulative_returns = (1 + weighted_returns).cumprod()
            
            # Annual returns
            annual_returns = cumulative_returns.resample("YE").last().pct_change().dropna()
            print("Annual Returns for the Portfolio:")
            print(annual_returns)
            
            # Total cumulative return
            total_cumulative_return = cumulative_returns.iloc[-1] - 1
            print(f"\nTotal Cumulative Return: {total_cumulative_return:.2%}")
            
            # Average annual return
            average_annual_return = (cumulative_returns.iloc[-1]) ** (1 / 5) - 1
            print(f"\nAverage Annual Return: {average_annual_return:.2%}")
            
            # Portfolio standard deviation (annualized)
            portfolio_std = weighted_returns.std() * np.sqrt(252)
            print(f"\nPortfolio Standard Deviation: {portfolio_std:.2%}")
            
            # Risk-free rate (using 3-month US Treasury Bill rate as a proxy)
            # Note: This is a simplified approach. In practice, you'd use the actual 
            # risk-free rate for the specific time period
            risk_free_rate = 0.02  # 2% annual risk-free rate
            
            # Sharpe Ratio calculation
            sharpe_ratio = (average_annual_return - risk_free_rate) / portfolio_std
            print(f"\nSharpe Ratio: {sharpe_ratio:.2f}")
            
        except KeyError as e:
            print(f"Error: {e}")
            print("Downloaded data columns:")
            print(data.columns)
    else:
        print("Error: 'Close' data not found.")
        if data is not None:
            print("Downloaded data columns:")
            print(data.columns)
        else:
            print("Data returned None")
else:
    print("Error: Data download failed.")

[*********************100%***********************]  5 of 5 completed

Annual Returns for the Portfolio:
Date
2020-12-31    0.916691
2021-12-31    0.520011
2022-12-31   -0.512295
2023-12-31    1.149191
Freq: YE-DEC, dtype: float64

Total Cumulative Return: 469.02%

Average Annual Return: 41.59%

Portfolio Standard Deviation: 41.69%

Sharpe Ratio: 0.95


In [204]:
import yfinance as yf
import pandas as pd
import numpy as np

# Portfolio parameters
tickers = ["GBTC", "SPY", "BRK-A", "VUG", "VOO"]
weights = [0, 0.25, 0.75, 0, 0]
start_date = "2019-01-01"
end_date = "2023-12-31"

# Download data
data = yf.download(tickers, start=start_date, end=end_date)

if data is not None and len(data) > 0:
    if "Close" in data:
        try:
            # Extract closing prices
            data = data["Close"]
            
            # Calculate returns
            returns = data.pct_change().dropna()
            
            # Calculate weighted portfolio returns
            weighted_returns = (returns * weights).sum(axis=1)
            
            # Cumulative returns calculation
            cumulative_returns = (1 + weighted_returns).cumprod()
            
            # Annual returns
            annual_returns = cumulative_returns.resample("YE").last().pct_change().dropna()
            print("Annual Returns for the Portfolio:")
            print(annual_returns)
            
            # Total cumulative return
            total_cumulative_return = cumulative_returns.iloc[-1] - 1
            print(f"\nTotal Cumulative Return: {total_cumulative_return:.2%}")
            
            # Average annual return
            average_annual_return = (cumulative_returns.iloc[-1]) ** (1 / 5) - 1
            print(f"\nAverage Annual Return: {average_annual_return:.2%}")
            
            # Portfolio standard deviation (annualized)
            portfolio_std = weighted_returns.std() * np.sqrt(252)
            print(f"\nPortfolio Standard Deviation: {portfolio_std:.2%}")
            
            # Risk-free rate (using 3-month US Treasury Bill rate as a proxy)
            # Note: This is a simplified approach. In practice, you'd use the actual 
            # risk-free rate for the specific time period
            risk_free_rate = 0.02  # 2% annual risk-free rate
            
            # Sharpe Ratio calculation
            sharpe_ratio = (average_annual_return - risk_free_rate) / portfolio_std
            print(f"\nSharpe Ratio: {sharpe_ratio:.2f}")
            
        except KeyError as e:
            print(f"Error: {e}")
            print("Downloaded data columns:")
            print(data.columns)
    else:
        print("Error: 'Close' data not found.")
        if data is not None:
            print("Downloaded data columns:")
            print(data.columns)
        else:
            print("Data returned None")
else:
    print("Error: Data download failed.")

[*********************100%***********************]  5 of 5 completed

Annual Returns for the Portfolio:
Date
2020-12-31    0.697984
2021-12-31    0.309216
2022-12-31   -0.372259
2023-12-31    0.764489
Freq: YE-DEC, dtype: float64

Total Cumulative Return: 273.45%

Average Annual Return: 30.15%

Portfolio Standard Deviation: 29.58%

Sharpe Ratio: 0.95


In [21]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import time

def scrape_vug_holdings():
    # Set up Selenium with Chrome
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
    driver.get("https://investor.vanguard.com/investment-products/etfs/profile/vug#portfolio-composition")
    
    # Wait for dynamic content to load
    time.sleep(5)  
    
    all_holdings = []
    
    # Find the select element for pagination
    select_element = driver.find_element(By.ID, "equitySelectId")
    select_object = Select(select_element)
    
    # Get the number of pages
    options = select_object.options
    total_pages = len(options)
    
    for page in range(1, total_pages + 1):
        print(f"Scraping page {page} of {total_pages}...")
        
        # Store the first ticker on the current page to detect changes
        current_first_ticker = driver.find_element(By.CSS_SELECTOR, "table.table tbody tr:first-child th[data-rpa-tag-id='symbol']").text.strip()
        
        # Select the page
        select_object.select_by_value(str(page))
        
        # Wait for page content to change
        # Instead of checking for staleness, wait for a specific timeout
        time.sleep(3)
        
        # For pages after the first one, verify the page actually changed by checking if the first ticker changed
        if page > 1:
            # Try multiple times to confirm page changed
            max_tries = 5
            for attempt in range(max_tries):
                try:
                    new_first_ticker = driver.find_element(By.CSS_SELECTOR, "table.table tbody tr:first-child th[data-rpa-tag-id='symbol']").text.strip()
                    if new_first_ticker != current_first_ticker:
                        # Page changed successfully
                        break
                    else:
                        print(f"Page didn't change on attempt {attempt+1}, trying again...")
                        select_object.select_by_value(str(page))
                        time.sleep(2)
                except:
                    time.sleep(1)
                    
                # If we've tried max times and still haven't changed, log it
                if attempt == max_tries - 1:
                    print(f"WARNING: Page {page} may not have loaded correctly!")
        
        # Extract table rows
        rows = driver.find_elements(By.CSS_SELECTOR, "table.table tbody tr")
        
        page_data = []  # Store this page's data separately for verification
        
        for row in rows:
            try:
                # Extract data using attributes
                ticker = row.find_element(By.CSS_SELECTOR, "th[data-rpa-tag-id='symbol']").text.strip()
                name = row.find_element(By.CSS_SELECTOR, "td[data-rpa-tag-id='holdings']").text.strip()
                weight = row.find_element(By.CSS_SELECTOR, "td[data-rpa-tag-id='fundPercent']").text.strip()
                shares = row.find_element(By.CSS_SELECTOR, "td[data-rpa-tag-id='shares']").text.strip()
                market_value = row.find_element(By.CSS_SELECTOR, "td[data-rpa-tag-id='marketValue']").text.strip()
                
                holding_data = {
                    "Ticker": ticker,
                    "Holdings": name,
                    "Weight": weight,
                    "Shares": shares,
                    "Market Value": market_value,
                    "Page": page  # Add page number for debugging
                }
                
                all_holdings.append(holding_data)
                page_data.append(holding_data)
                
            except Exception as e:
                print(f"Skipping a row due to error: {e}")
                continue
        
        print(f"Found {len(page_data)} holdings on page {page}")
        # Print first ticker on this page for verification
        if page_data:
            print(f"First ticker on page {page}: {page_data[0]['Ticker']}")
    
    # Close the browser
    driver.quit()
    
    # Save to CSV
    df = pd.DataFrame(all_holdings)
    
    # Check for duplicates (potential indicator of pagination issues)
    duplicate_count = df[df.duplicated(['Ticker', 'Holdings'])].shape[0]
    if duplicate_count > 0:
        print(f"WARNING: Found {duplicate_count} potential duplicate entries!")
    
    df.to_csv("VUG_Holdings_Full.csv", index=False)
    print(f"Success! Saved {len(df)} holdings to 'VUG_Holdings_Full.csv'.")
    
    # Print summary statistics
    print(f"Total holdings collected: {len(df)}")
    page_counts = df['Page'].value_counts().sort_index()
    for page_num, count in page_counts.items():
        print(f"Page {page_num}: {count} holdings")
    
    return df

# Run the scraper
vug_holdings = scrape_vug_holdings()
vug_holdings.head()  # Preview the data

Scraping page 1 of 18...
Skipping a row due to error: Message: no such element: Unable to locate element: {"method":"css selector","selector":"th[data-rpa-tag-id='symbol']"}
  (Session info: chrome=134.0.6998.166); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
0   chromedriver                        0x0000000102d376c8 cxxbridge1$str$ptr + 2791212
1   chromedriver                        0x0000000102d2fc9c cxxbridge1$str$ptr + 2759936
2   chromedriver                        0x0000000102881e30 cxxbridge1$string$len + 92928
3   chromedriver                        0x00000001028c9170 cxxbridge1$string$len + 384576
4   chromedriver                        0x00000001028be818 cxxbridge1$string$len + 341224
5   chromedriver                        0x000000010290a5f8 cxxbridge1$string$len + 651976
6   chromedriver                        0x00000001028bd2fc cxxbridge1$string$len + 335820
7  

,Ticker,Holdings,Weight,Shares,Market Value,Page
0,AAPL,Apple Inc.,13.09 %,"153,056,589","$37,015,205,484",1
1,MSFT,Microsoft Corp.,10.57 %,"75,288,105","$29,888,624,804",1
2,NVDA,NVIDIA Corp.,10.40 %,"235,593,495","$29,430,339,395",1
3,AMZN,Amazon.com Inc.,7.19 %,"95,868,270","$20,350,916,356",1
4,META,Facebook Inc. Class A,5.22 %,"22,098,466","$14,766,194,981",1
